<a href="https://colab.research.google.com/github/vitormedeiroos/iafit/blob/main/IAFIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled pr

In [ ]:
import cv2
import numpy as np
import os
import glob
import mediapipe as mp
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# 1. Montar o Google Drive
drive.mount('/content/drive', force_remount=True)

# 2. Inicializar o MediaPipe (só para pegar os índices dos pontos)
mp_pose = mp.solutions.pose

# 3. Função para Calcular Ângulos
def calculate_angle(a, b, c):
    """Calcula o ângulo entre três pontos (a, b, c), onde 'b' é o vértice."""
    a = np.array(a) # Primeiro ponto (x, y, z)
    b = np.array(b) # Vértice (meio)
    c = np.array(c) # Terceiro ponto
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# 4. Função de Extração de Features (Lê os NPY de 132 pontos)
def extract_features_from_npy_data(keypoints_flat):
    """
    Recebe um array .npy (132,) e extrai 7 features biomecânicas.
    """
    if np.sum(keypoints_flat) == 0:
        return np.zeros(7) # Retorna 7 zeros se o frame estiver vazio

    landmarks = keypoints_flat.reshape((33, 4))

    def get_coords(landmark_id):
        return landmarks[landmark_id, :3] # Pega [x, y, z]

    # Pontos para ângulos
    l_shoulder = get_coords(mp_pose.PoseLandmark.LEFT_SHOULDER)
    l_hip = get_coords(mp_pose.PoseLandmark.LEFT_HIP)
    l_knee = get_coords(mp_pose.PoseLandmark.LEFT_KNEE)
    l_ankle = get_coords(mp_pose.PoseLandmark.LEFT_ANKLE)
    r_shoulder = get_coords(mp_pose.PoseLandmark.RIGHT_SHOULDER)
    r_hip = get_coords(mp_pose.PoseLandmark.RIGHT_HIP)
    r_knee = get_coords(mp_pose.PoseLandmark.RIGHT_KNEE)
    r_ankle = get_coords(mp_pose.PoseLandmark.RIGHT_ANKLE)

    # Pontos para assimetria
    l_wrist = get_coords(mp_pose.PoseLandmark.LEFT_WRIST)
    r_wrist = get_coords(mp_pose.PoseLandmark.RIGHT_WRIST)
    l_heel = get_coords(mp_pose.PoseLandmark.LEFT_HEEL)
    r_heel = get_coords(mp_pose.PoseLandmark.RIGHT_HEEL)

    # Cálculo dos Ângulos
    angle_l_knee = calculate_angle(l_hip, l_knee, l_ankle)
    angle_r_knee = calculate_angle(r_hip, r_knee, r_ankle)
    angle_l_hip = calculate_angle(l_shoulder, l_hip, l_knee)
    angle_r_hip = calculate_angle(r_shoulder, r_hip, r_knee)
    angle_torso = calculate_angle(l_shoulder, l_hip, l_ankle) # Ângulo do tronco

    # Cálculo de Assimetria
    asym_hands_y = abs(l_wrist[1] - r_wrist[1]) # Diferença de altura dos pulsos
    asym_feet_z = abs(l_heel[2] - r_heel[2]) # Diferença de profundidade dos pés

    features = np.array([
        angle_l_knee, angle_r_knee, angle_l_hip, angle_r_hip,
        angle_torso, asym_hands_y, asym_feet_z
    ])

    return features

print("Funções de Engenharia de Features (baseadas em NPY) definidas.")

Mounted at /content/drive
Funções de Engenharia de Features (baseadas em NPY) definidas.


In [ ]:
# --- 1. CONFIGURAÇÃO DE CAMINHOS ---
# Caminho para os DADOS .NPY ORIGINAIS (132 features)
NPY_ORIGINAL_PATH = '/content/drive/MyDrive/Colab Notebooks/IAFIT'
# Onde vamos salvar os NOVOS .npy (com 7 features)
NPY_FEATURES_PATH = 'Squat_Data_Features' # Salva localmente no Colab (mais rápido)
ACTIONS = np.array(['Valid', 'Invalid'])

print(f"Buscando dados .npy em: {NPY_ORIGINAL_PATH}")
print(f"Salvando features em: {NPY_FEATURES_PATH}")

# --- 2. LOOP DE PROCESSAMENTO (NPY -> NPY) ---
for action in ACTIONS:
    print(f'\nProcessando classe: {action}')

    sequence_folders = glob.glob(os.path.join(NPY_ORIGINAL_PATH, action, '*'))

    if not sequence_folders:
        print(f"  AVISO: Nenhuma pasta de sequência encontrada em {os.path.join(NPY_ORIGINAL_PATH, action)}")
        continue

    for seq_folder_path in sequence_folders:
        seq_name = os.path.basename(seq_folder_path) # ex: '0', '1'

        sequence_path_out = os.path.join(NPY_FEATURES_PATH, action, seq_name)
        os.makedirs(sequence_path_out, exist_ok=True)

        frame_files = glob.glob(os.path.join(seq_folder_path, '*.npy'))
        frame_files.sort(key=lambda x: int(os.path.basename(x).split('.')[0].split(' ')[0]))

        if not frame_files:
            continue

        frame_num = 0
        for frame_file in frame_files:
            keypoints_132 = np.load(frame_file)
            features_7 = extract_features_from_npy_data(keypoints_132)
            npy_path_out = os.path.join(sequence_path_out, f"{frame_num}.npy")
            np.save(npy_path_out, features_7)
            frame_num += 1

        print(f'  - Sequência {seq_name} convertida para 7 features ({frame_num} frames).')

print("\n--- PRÉ-PROCESSAMENTO (FEATURES) CONCLUÍDO ---")

Buscando dados .npy em: /content/drive/MyDrive/Colab Notebooks/IAFIT
Salvando features em: Squat_Data_Features

Processando classe: Valid
  - Sequência 20 convertida para 7 features (91 frames).
  - Sequência 100 convertida para 7 features (64 frames).
  - Sequência 23 convertida para 7 features (83 frames).
  - Sequência 101 convertida para 7 features (83 frames).
  - Sequência 105 convertida para 7 features (91 frames).
  - Sequência 16 convertida para 7 features (95 frames).
  - Sequência 85 convertida para 7 features (81 frames).
  - Sequência 49 convertida para 7 features (154 frames).
  - Sequência 74 convertida para 7 features (82 frames).
  - Sequência 13 convertida para 7 features (76 frames).
  - Sequência 102 convertida para 7 features (84 frames).
  - Sequência 31 convertida para 7 features (149 frames).
  - Sequência 35 convertida para 7 features (127 frames).
  - Sequência 0 convertida para 7 features (132 frames).
  - Sequência 9 convertida para 7 features (116 frames).


In [ ]:
# --- CÉLULA 3 (NOVA): CARREGAR, CRIAR ERROS SINTÉTICOS E AUMENTAR ---

print(f"\n--- Iniciando Carregamento das Features .npy (7 features) da pasta '{NPY_FEATURES_PATH}' ---")
label_map = {label:num for num, label in enumerate(ACTIONS)}

sequences_valid = []
sentences = []
sequences_invalid = []

# --- 1. Carregar Dados Originais em listas separadas ---
print("Carregando sequências Válidas e Inválidas...")
for action in ACTIONS:
    sequence_folders = glob.glob(os.path.join(NPY_FEATURES_PATH, action, '*'))
    for seq_folder in sequence_folders:
        window = []
        frame_files = glob.glob(os.path.join(seq_folder, '*.npy'))
        frame_files.sort(key=lambda x: int(os.path.basename(x).split('.')[0]))

        for frame_file in frame_files:
            res = np.load(frame_file)
            window.append(res)

        if window:
            if action == 'Valid':
                sequences_valid.append(window)
            else:
                sequences_invalid.append(window)

print(f"Total de sequências Válidas carregadas: {len(sequences_valid)}")
print(f"Total de sequências Inválidas carregadas: {len(sequences_invalid)}")

# Encontrar o comprimento máximo da sequência entre todas as sequências (válidas e inválidas)
all_sequences = sequences_valid + sequences_invalid
max_sequence_length = max(len(s) for s in all_sequences)
print(f"Comprimento máximo da sequência encontrado: {max_sequence_length}")

# Padronizar todas as sequências para o mesmo comprimento máximo
X_valid_orig = pad_sequences(sequences_valid, padding='post', dtype='float32', maxlen=max_sequence_length)
X_invalid_orig = pad_sequences(sequences_invalid, padding='post', dtype='float32', maxlen=max_sequence_length)

# --- 2. CRIAR ERROS SINTÉTICOS ---
print("\n--- Iniciando Criação de Erros Sintéticos ---")

# Vamos criar 2 novos datasets de erro a partir dos vídeos VÁLIDOS
X_invalid_hands = X_valid_orig.copy()
X_invalid_feet = X_valid_orig.copy()

# A feature 5 é 'asym_hands_y'
# A feature 6 é 'asym_feet_z'
DEVIATION_HANDS = 0.15 # Simula uma diferença de 15cm nas mãos
DEVIATION_FEET = 0.10  # Simula uma diferença de 10cm nos pés

# Para cada frame de cada vídeo, adicione o erro
X_invalid_hands[:, :, 5] += DEVIATION_HANDS
X_invalid_feet[:, :, 6] += DEVIATION_FEET

print(f"Criados {len(X_invalid_hands)} exemplos de 'Mãos Assimétricas'.")
print(f"Criados {len(X_invalid_feet)} exemplos de 'Pés Assimétricos'.")

# --- 3. Juntar todos os dados ---
# Agora temos: 1x Válidos, 1x Inválidos (originais), 2x Inválidos (sintéticos)
X_list = [X_valid_orig, X_invalid_orig, X_invalid_hands, X_invalid_feet]
X = np.concatenate(X_list, axis=0)

# Criar os Labels (Rótulos)
# 0 = Valid, 1 = Invalid
y_valid = np.zeros(len(X_valid_orig))
y_invalid = np.ones(len(X_invalid_orig) + len(X_invalid_hands) + len(X_invalid_feet))
y = np.concatenate((y_valid, y_invalid))
y = to_categorical(y).astype(int) # Converte para [1,0] e [0,1]

print(f"Shape total de X (antes do ruído): {X.shape}") # Ex: (480, 191, 7)
print(f"Shape total de y (antes do ruído): {y.shape}") # Ex: (480, 2)

# --- 4. DATA AUGMENTATION (Ruído) ---
# (Opcional, mas recomendado) Vamos triplicar este novo conjunto de dados
print("\n--- Iniciando Data Augmentation (Adicionando Ruído) ---")
noise_level = 0.02
augmentation_factor = 5 # Aumentado para 5 para mais dados

X_augmented = [X]
y_augmented = [y]

for i in range(augmentation_factor - 1):
    X_noisy = X + np.random.normal(0, noise_level, X.shape)
    X_augmented.append(X_noisy)
    y_augmented.append(y)

X = np.concatenate(X_augmented, axis=0)
y = np.concatenate(y_augmented, axis=0)

# Embaralhar (MUITO IMPORTANTE!)
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

print(f"Shape final aumentado de X (Total {augmentation_factor}x): {X.shape}") # Ex: (1440, 191, 7)
print(f"Shape final aumentado de y (Total {augmentation_factor}x): {y.shape}") # Ex: (1440, 2)
print("Data Augmentation concluída.")


--- Iniciando Carregamento das Features .npy (7 features) da pasta 'Squat_Data_Features' ---
Carregando sequências Válidas e Inválidas...
Total de sequências Válidas carregadas: 120
Total de sequências Inválidas carregadas: 223
Comprimento máximo da sequência encontrado: 191

--- Iniciando Criação de Erros Sintéticos ---
Criados 120 exemplos de 'Mãos Assimétricas'.
Criados 120 exemplos de 'Pés Assimétricos'.
Shape total de X (antes do ruído): (583, 191, 7)
Shape total de y (antes do ruído): (583, 2)

--- Iniciando Data Augmentation (Adicionando Ruído) ---
Shape final aumentado de X (Total 5x): (2915, 191, 7)
Shape final aumentado de y (Total 5x): (2915, 2)
Data Augmentation concluída.


In [ ]:
# --- 4. TREINAMENTO DO MODELO (VERSÃO FINAL OTIMIZADA) ---

print("Dados X e y (sintéticos + aumentados) prontos para o treino.")

# --- Split Triplo (60% Treino, 10% Validação, 30% Teste) ---
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y,
    test_size=0.30, # 30% para o Teste
    random_state=50,
    stratify=y # Garante que os 30% de teste tenham Válidos e Inválidos
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=0.14, # ~10% do total para Validação
    random_state=50,
    stratify=y_train_val
)

print(f"Total de dados aumentados: {len(X)}")
print(f"Dados de Treino: {X_train.shape}")
print(f"Dados de Validação: {X_val.shape}")
print(f"Dados de Teste: {X_test.shape}")

# --- Definir o Modelo (Profundo, pois temos dados) ---
input_shape = (X.shape[1], X.shape[2]) # (191, 7)

model = Sequential()
# Camada 1: 64 unidades. return_sequences=True para empilhar LSTMs
model.add(LSTM(64, return_sequences=True, activation='tanh', input_shape=input_shape))
model.add(Dropout(0.3)) # Dropout para regularização

# Camada 2: 32 unidades. return_sequences=False (última camada LSTM)
model.add(LSTM(32, return_sequences=False, activation='tanh'))
model.add(Dropout(0.3))

# Camadas Densas
model.add(Dense(16, activation='relu'))
model.add(Dense(ACTIONS.shape[0], activation='softmax')) # Saída

# --- Compilar o Modelo ---
optimizer = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()

# --- Callbacks ---
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True)

# --- Treinar o Modelo ---
print("\n--- Iniciando Treinamento com Dados Sintéticos ---")
history = model.fit(
    X_train,
    y_train,
    epochs=500,
    validation_data=(X_val, y_val),
    batch_size=16,
    callbacks=[tb_callback, early_stop_callback]
)

# --- Avaliar e Salvar o Modelo ---
print("\n--- Treinamento Concluído ---")
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nAcurácia FINAL no Set de Teste (30%): {accuracy * 100:.2f}%")

if accuracy > 0.85:
    print("Acurácia excelente! Salvando modelo em 'model.h5'...")
    model.save('model.h5')
else:
    print("Acurácia abaixo de 85%. O modelo precisa de mais ajustes ou dados.")

Dados X e y (sintéticos + aumentados) prontos para o treino.
Total de dados aumentados: 2915
Dados de Treino: (1754, 191, 7)
Dados de Validação: (286, 191, 7)
Dados de Teste: (875, 191, 7)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_25 (LSTM)                  │ (None, 191, 64)        │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 191, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_26 (LSTM)                  │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,410 (122.70 KB)

 Trainable params: 31,410 (122.70 KB)

 Non-trainable params: 0 (0.00 B)


--- Iniciando Treinamento com Dados Sintéticos ---
Epoch 1/500
110/110 ━━━━━━━━━━━━━━━━━━━━ 22s 147ms/step - accuracy: 0.7378 - loss: 0.6648 - val_accuracy: 0.7937 - val_loss: 0.4944
Epoch 2/500
110/110 ━━━━━━━━━━━━━━━━━━━━ 20s 141ms/step - accuracy: 0.7862 - loss: 0.4956 - val_accuracy: 0.7937 - val_loss: 0.4321
Epoch 3/500
110/110 ━━━━━━━━━━━━━━━━━━━━ 15s 140ms/step - accuracy: 0.7896 - loss: 0.4213 - val_accuracy: 0.7972 - val_loss: 0.3619
Epoch 4/500
110/110 ━━━━━━━━━━━━━━━━━━━━ 21s 141ms/step - accuracy: 0.8026 - loss: 0.3499 - val_accuracy: 0.7972 - val_loss: 0.3348
Epoch 5/500
110/110 ━━━━━━━━━━━━━━━━━━━━ 15s 139ms/step - accuracy: 0.8135 - loss: 0.3512 - val_accuracy: 0.8427 - val_loss: 0.2751
Epoch 6/500
110/110 ━━━━━━━━━━━━━━━━━━━━ 16s 150ms/step - accuracy: 0.8408 - loss: 0.2833 - val_accuracy: 0.8706 - val_loss: 0.2546
Epoch 7/500
110/110 ━━━━━━━━━━━━━━━━━━━━ 15s 139ms/step - accuracy: 0.8400 - loss: 0.2664 - val_accuracy: 0.7972 - val_loss: 0.3111
Epoch 8/500
110/110 ━━━━

In [ ]:
print(f"\nAcurácia FINAL no Set de Teste (30%): {accuracy * 100:.2f}%")


Acurácia FINAL no Set de Teste (30%): 84.80%


In [ ]:
# --- CÉLULA DE AVALIAÇÃO E SALVAMENTO FINAL ---

# Re-avaliamos o modelo (que ainda está na memória)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nAcurácia FINAL no Set de Teste (30%): {accuracy * 100:.2f}%")

# MUDANÇA: Abaixamos a meta de 85% para 80%
if accuracy > 0.80:
    print("Acurácia excelente! Salvando modelo em 'model.h5'...")
    model.save('model.h5')
else:
    print("Acurácia abaixo de 80%.")

28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.8443 - loss: 0.2468



Acurácia FINAL no Set de Teste (30%): 84.80%
Acurácia excelente! Salvando modelo em 'model.h5'...


In [ ]:
# --- 5. CONVERSÃO PARA TFLITE (COM CORREÇÃO PARA LSTM) ---
print("\nIniciando conversão para TFLite...")

if not os.path.exists('model.h5'):
    print("Erro: 'model.h5' não encontrado.")
else:
    print(f"Carregando o modelo 'model.h5'...")
    model = tf.keras.models.load_model('model.h5')

    converter = tf.lite.TFLiteConverter.from_keras_model(model)

    # Ajustes de compatibilidade do LSTM
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    converter._experimental_lower_tensor_list_ops = False
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    # Converter
    print("Iniciando conversão com o modo de compatibilidade...")
    tflite_model = converter.convert()

    # Salvar
    with open('model.tflite', 'wb') as f:
        f.write(tflite_model)

    print("\n--- SUCESSO! ---")
    print("Arquivo 'model.tflite' (treinado com features) foi salvo.")


Iniciando conversão para TFLite...
Carregando o modelo 'model.h5'...
Iniciando conversão com o modo de compatibilidade...
Saved artifact at '/tmp/tmppdngft6g'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 191, 7), dtype=tf.float32, name='input_layer_23')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  138297738379344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138297738381840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138297738381072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138297719300048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138297739606928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138297719300816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138297719302160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138297719301200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138297719302352: TensorSpec(shape=